<a href="https://colab.research.google.com/github/THEFLILUX/GG_SEARCH_BD2/blob/main/Backend/InvertedIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import io
from google.colab import files

teacher_sw = files.upload()

Saving stoplist_new.txt to stoplist_new.txt


*Function to get the words from the file and removes the stop words from stoplist.txt and other unnecessary characters*

In [4]:
def getStopWordsFromFile(filename):
  stoplist = open(filename, encoding='latin-1')

  stop_words = []
  for line in stoplist:
    line = line.rstrip('\r\n')
    stop_words.append(line);
  
  if stoplist:
    stoplist.close()

  #additionals
  stop_words.append(",")
  stop_words.append(".")
  stop_words.append(";")
  stop_words.append("La")
  stop_words.append("la")
  stop_words.append("«")
  stop_words.append("»")
  stop_words.append(":")
  stop_words.append("El")
  stop_words.append("En")
  stop_words.append("De")
  stop_words.append("Al")
  stop_words.append("(")
  stop_words.append(")")
  stop_words.append("``")
  stop_words.append("''")
  stop_words.append("“")
  stop_words.append("”")
  stop_words.append("...")
  stop_words.append("RT")
  stop_words.append("--")
  stop_words.append(".-")
  stop_words.append("..")
  stop_words.append("*-*")
  stop_words.append("***")
  stop_words.append("/…")
  return stop_words

In [5]:
import re

emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U0001F914-\U0001F914"  #Thinking emoticon 🤔
                           u"\U0001f924-\U0001f924"  #🤤
                           u"\U0001F923-\U0001F923" #🤣
                           u"\U0001F922-\U0001F922" #🤢
                           u"\U00002702-\U000027B0" 
                           u"\U000024C2-\U0001F251"
                           u"\U0001F997-\U0001F997" #🦗 
                           "]+", flags=re.UNICODE)

*Function to remove the stop words from a token list*

In [30]:
def remove_stop_words(token_list, sw_list):
  tokens_clean = token_list.copy()
  for token in token_list:
    if token in sw_list:
      tokens_clean.remove(token)
    #remove emojis
    tmp = emoji_pattern.sub(r'', token)    
    if len(token) != len(tmp):
      tokens_clean.remove(token)

    if "//t.co/" in token:
      try:     
        tokens_clean.remove(token)
      except ValueError as e:
        print(" Error in token ", token)

  return tokens_clean

In [7]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
class WordCount:
  def __init__(self):
    self.df = set()
    self.tfd = {}


In [9]:
def calculate_df_tfd(tokens_clean, terms, doc_id):
  from nltk.stem import SnowballStemmer
  stemmer = SnowballStemmer('spanish')

  for token in tokens_clean:
    token = stemmer.stem(token)
    if token not in terms:
      terms[token] = WordCount() 
      
      terms[token].df.add(doc_id)

      if doc_id in terms[token].tfd:
        terms[token].tfd[doc_id] = terms[token].tfd[doc_id] + 1
      else:
        terms[token].tfd[doc_id] = 1

    else:
      tmp = terms[token]
      
      tmp.df.add(doc_id)

      if doc_id in tmp.tfd:
        tmp.tfd[doc_id] = tmp.tfd[doc_id] + 1
      else:
        tmp.tfd[doc_id] = 1

      terms[token] = tmp

###Formula

**w𝑡,𝑑=log10(1+tf𝑡,𝑑)×log10(𝑁/df𝑡)**

In [10]:
import math 
from decimal import Decimal

def calculate_tf_idf(N, df, tfd):
  a = math.log((1+tfd), 10)
  b = math.log((N/df), 10)
  return round((a * b), 6)

In [28]:
import math 

def save_norm(filename, norm_dict):
  
  norm_sorted = sorted(norm_dict)

  with open(filename, 'w') as writefile:
    line = ""
    for key in norm_sorted:
      tmp = math.sqrt(norm_dict[key])
      tmp = round(tmp, 6)
      line = str(key) + " " + str(tmp) + "\n"
      writefile.write(line)
  print(filename + " saved ")

In [34]:
import os
import json
import nltk 
from nltk.tokenize import word_tokenize
#Getting the stop words
stoplist = getStopWordsFromFile('stoplist_new.txt')

path = '/content/drive/My Drive/DB2/CleanData'
json_list = os.listdir(path)

tfid_File = "tfidf.txt"
norm_file = "normas.txt"
print("There are ", len(json_list), " json files")

terms = {}
N_count = 0
for file_name in json_list:
#with open('/content/drive/My Drive/DB2/CleanData/tweets_2018-08-07.json', 'r') as f:
#file_name = "tweets_2018-08-30.json"
  if file_name == tfid_File or file_name == norm_file:
    continue
     
  print("Processing: ", file_name)
  with open(path + "/" + file_name, 'r') as f:
    tmp = json.load(f)
    for json_str in tmp:
            #print(json_str)
      docId = json_str['id']
      textAttr = json_str["text"]
      tokens = nltk.word_tokenize(textAttr) #Tokenization
          #print(tokens)
      N_count = N_count + 1
      tokens_clean = remove_stop_words(tokens, stoplist) #removing stop words
            #print(docId, " ", len(tokens), " ", len(tokens_clean))
            #print(docId, " ", tokens)
      if tokens_clean:
              #print(docId, " ", tokens_clean)
        calculate_df_tfd(tokens_clean, terms, docId)

#Save in secondary memory

#tfidf file
terms_sorted = sorted(terms)

norm = {}
with open(path + "/" + tfid_File, 'w') as writefile:
  for key in terms_sorted:
    line = ""
    obj = terms[key]
    #print(key, len(obj.df))
       
    for docId in obj.tfd:
      #line = line + "," + str(tfi) + ":" + str(obj.tfd[tfi])
      rate = calculate_tf_idf(N_count, len(obj.df), obj.tfd[docId])
      line = line + str(tfi) + "," + str(rate) + ":"
      
      if docId in norm:
        tmp = rate*rate
        norm[docId] = norm[docId] + tmp        
      else:
        norm[docId] = rate * rate

    line = key + ":" + line[:-1] + "\n"
    writefile.write(line)
    #print(line)

#Norm file
filename = path + "/" + norm_file
save_norm(filename, norm)

There are  59  json files
Processing:  tweets_2018-08-31.json
Processing:  tweets_2018-08-12.json
Processing:  tweets_2018-08-10.json
Processing:  tweets_2018-08-11.json
Processing:  tweets_2018-08-09.json
Processing:  tweets_2018-08-08.json
Processing:  tweets_2018-08-07.json
Processing:  tweets_2018-10-02.json
Processing:  tweets_2018-10-03.json
Processing:  tweets_2018-10-04.json
Processing:  tweets_2018-09-28.json
Processing:  tweets_2018-10-01.json
Processing:  tweets_2018-10-05.json
Processing:  tweets_2018-09-25.json
Processing:  tweets_2018-09-27.json
Processing:  tweets_2018-09-26.json
Processing:  tweets_2018-09-29.json
Processing:  tweets_2018-09-22.json
Processing:  tweets_2018-09-20.json
Processing:  tweets_2018-09-21.json
Processing:  tweets_2018-09-30.json
Processing:  tweets_2018-09-16.json
Processing:  tweets_2018-09-24.json
Processing:  tweets_2018-09-18.json
Processing:  tweets_2018-09-19.json
Processing:  tweets_2018-09-17.json
Processing:  tweets_2018-09-15.json
Pr